In [1]:
import json
import copy
import cv2
import numpy as np


In [2]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    chunks_lst = []
    for i in range(0, len(lst), n):
         chunks_lst.append(lst[i : i + n])
    return chunks_lst


In [3]:
def find_extrema_rectangle(points):
    """Find the extrema of a polygon and return rectangle coordinates."""
    x = [point[0] for point in points]
    y = [point[1] for point in points]
    x_min, x_max = min(x), max(x)
    y_min, y_max = min(y), max(y)
    return [(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)]
    

In [4]:
# find_extrema_rectangle(chunks(shapes_by_frame[0][0]["points"], 2))

In [5]:
def intersect(polygon1, polygon2):
    """Check if two polygons intersect"""
    polygon1 = np.array(polygon1, dtype=np.int32)
    polygon2 = np.array(polygon2, dtype=np.int32)
    basic_img = np.zeros([1000, 1000], dtype=np.uint8)
    red_poly = basic_img.copy()
    cv2.fillPoly(red_poly, [polygon1], 1)
    blue_poly = basic_img.copy()
    cv2.fillPoly(blue_poly, [polygon2], 1)
    intersection = red_poly * blue_poly
    return np.any(intersection)


In [6]:
def assign_id(shapes_by_frame: list[list]):
    """
    Assigns id to each shape in the list of shapes grouped by frame
    ID is assigned according to the position of the shape in the frame
    """
    id_count = 0
    # assign id to shapes in the first frame
    for i, shape in enumerate(shapes_by_frame[0]):
        shape["id"] = i
        id_count += 1

    # assign id to shapes in the rest of the frames
    for i, frame in enumerate(shapes_by_frame[1:], start=1):
        for shape in frame:
            # check if the shape with the same id is present in the previous frame
            # if same label and square intersect: then same id else: new id
            for prev_shape in shapes_by_frame[i-1]:
                if shape["label"] == prev_shape["label"] and intersect(
                    find_extrema_rectangle(chunks(shape["points"], 2)),
                    find_extrema_rectangle(chunks(prev_shape["points"], 2))
                ):
                    shape["id"] = prev_shape["id"]
                    break
            else:
                shape["id"] = id_count
                id_count += 1


In [24]:
with open("test/orig/task_sonoscape_2021-09_2021-10-18_002_20211008_110924_87.avi_backup_2022_04_01_08_48_32 (1)/annotations.json") as json_file:
    annots = json.load(json_file)

shapes = copy.deepcopy(annots[0]["shapes"])
# filter shapes
filtered_shapes = [
    shape
    for shape in shapes
    if shape["type"] == "polygon"
    and shape["label"] in ["aline"]
]
# group shapes by frame
frame = 0
last_frame = filtered_shapes[-1]["frame"]
shapes_by_frame = []
shapes_in_single_frame = []
for i, shape in enumerate(filtered_shapes):
    if shape["frame"] == frame:
        shapes_in_single_frame.append(shape)
        # if multiple shapes in the last frame, add to shapes_by_frame
        if i == len(filtered_shapes) - 1:
            shapes_by_frame.append(shapes_in_single_frame)

    else:
        frame = frame + 10 if (frame + 10) < last_frame else last_frame
        shapes_by_frame.append(shapes_in_single_frame)
        shapes_in_single_frame = []
        shapes_in_single_frame.append(shape)


In [23]:
print(len(shapes_by_frame))

50


In [15]:
assign_id(shapes_by_frame)

In [25]:
# remove duplicate shapes in the same frame
for frame in shapes_by_frame:
    if len(frame) == 1:
        continue
    # find if two polygons intersect by comparing each other
    for i in range(len(frame)):
        for j in range(i + 1, len(frame)):
            try:
                if intersect(chunks(frame[i]["points"], 2), chunks(frame[j]["points"], 2)):
                    # remove one of the polygons
                    frame.pop(j)
            except:
                raise Exception("Error in frame: ")


In [17]:
num_of_lines = len(filtered_shapes)
len(filtered_shapes)


141

In [23]:
filtered_shapes[-1]["frame"]


285

In [24]:
filtered_shapes[0]


{'type': 'polygon',
 'occluded': False,
 'z_order': 0,
 'points': [88.0693359375,
  357.8681640625,
  183.30157275796046,
  334.05996236801184,
  285.4455657184135,
  339.43596199750937,
  277.76556624770274,
  358.63596067428625,
  184.8375726521026,
  357.0999607801441,
  81.9255797445785,
  376.299959456921],
 'frame': 0,
 'group': 0,
 'source': 'manual',
 'attributes': [],
 'label': 'lungslidingpresent',
 'id': 0}

In [25]:
print(len(shapes_by_frame))
print(len(shapes_by_frame[0][0]["points"]))
print(len(shapes_by_frame[1][0]["points"]))

29
12
12


In [26]:
len(shapes_by_frame[-1])
shapes_by_frame[-1]


[{'type': 'polygon',
  'occluded': False,
  'z_order': 0,
  'points': [84.607421875,
   356.5830078125,
   179.83965869546046,
   332.77480611801184,
   281.9836516559135,
   338.15080574750937,
   274.30365218520274,
   357.35080442428625,
   181.3756585896026,
   355.8148045301441,
   78.4636656820785,
   375.014803206921],
  'frame': 280,
  'group': 0,
  'source': 'manual',
  'attributes': [],
  'label': 'lungslidingpresent',
  'id': 0},
 {'type': 'polygon',
  'occluded': False,
  'z_order': 0,
  'points': [405.32421875,
   331.46826171875,
   550.4762587130081,
   319.94823274612463,
   548.1722588717948,
   339.9162313699726,
   418.3802678167831,
   349.1322307348255],
  'frame': 280,
  'group': 0,
  'source': 'manual',
  'attributes': [],
  'label': 'lungslidingpresent',
  'id': 1}]

In [27]:
def get_skelet(type: str):
    """Get the skeleton of the shape"""
    return {
        "type": "polygon",
        "occluded": False,
        "z_order": 0,
        "points": [],
        "frame": 0,
        "group": 0,
        "source": "manual",
        "attributes": [],
        "label": type,
    }


skelet = get_skelet("lungslidingpresent")


In [30]:
indicies = dict((d["frame"], index) for (index, d) in enumerate(shapes))
print(indicies)


{0: 2, 10: 4, 20: 6, 30: 8, 40: 10, 50: 12, 60: 14, 70: 16, 80: 18, 90: 20, 100: 22, 110: 24, 120: 26, 130: 28, 140: 30, 150: 32, 160: 34, 170: 36, 180: 38, 190: 40, 200: 42, 210: 44, 220: 46, 230: 48, 240: 50, 250: 52, 260: 54, 270: 56, 280: 58, 285: 60}


In [31]:
def add_substract_point(shapes: list[list[float]]):
    upper_left_idx = []
    add = True if len(shapes[0]) < len(shapes[1]) else False
    coords = np.array([list(chunks(shapes[0], 2)), list(chunks(shapes[1], 2))])
    for i in range(len(shapes)):
        magnitudes = [np.sqrt(x.dot(x)) for x in coords[i]]
        # get most upper left
        upper_left = min(magnitudes)
        # append index of the upper left coord to list of max 2 indicies
        upper_left_idx.append(magnitudes.index(upper_left))

    # value by which all points will be shifted
    shift = coords[1][upper_left_idx[1]] - coords[0][upper_left_idx[0]]
    # start polygons from the most upper left point
    if upper_left_idx[0] != 0:
        coords[0] = np.roll(coords[0], -upper_left_idx[0], axis=0)
    if upper_left_idx[1] != 0:
        coords[1] = np.roll(coords[1], -upper_left_idx[1], axis=0)
    # number of points to be added or substracted
    diff = abs(len(coords[0]) - len(coords[1]))
    # count distances between all points
    distances_in_first = [
        int(np.linalg.norm(coords[0][i] - coords[0][i + 1]))
        for i in range(len(coords[0]) - 1)
    ]
    distances_in_second = [
        int(np.linalg.norm(coords[1][i] - coords[1][i + 1]))
        for i in range(len(coords[1]) - 1)
    ]

    for _ in range(diff):
        for i in range(len(distances_in_first)):
            # compare differences between distances with treshold
            if abs(distances_in_first[i] - distances_in_second[i]) > 10:
                # add or substract point
                if add:
                    coords[0] = np.insert(
                        coords[0], i + 1, (coords[1][i + 1] - shift), axis=0
                    )
                else:
                    coords[0] = np.delete(coords[0], i + 1, axis=0)
                distances_in_first = [
                    int(np.linalg.norm(coords[0][i] - coords[0][i + 1]))
                    for i in range(len(coords[0]) - 1)
                ]
                break

    return coords[0].flatten().tolist()


In [46]:
add_substract_point([shapes_by_frame[0][0]["points"], shapes_by_frame[1][0]["points"]])


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [36]:
def interpolator(frames: list[dict]):
    """
    Interpolates between two frames
    If polygons have different number of vertices, it adds or substracts vertices

        Parameters:
            frames (list[dict]): shapes grouped by frames which they are present in
    """
    offset = 1
    # iterates over grouped by frame shapes
    for i, group in enumerate(frames[:-1]):
        # check if some shapes are missing in the next frame
        cut = max(0, len(group) - len(frames[i + 1]))
        # check which shapes are missing in the next frame and remove them from the group
        # shapes are checked by treir id
        if cut > 0:
            for _ in range(cut):
                for shape in group:
                    if shape["id"] not in [x["id"] for x in frames[i + 1]]:
                        group.remove(shape)
                        break
        frames_to_interpolate = frames[i + 1][0]["frame"] - frames[i][0]["frame"]
        difference, step = [], []
        # iterates over shapes in a group
        for j, shape in enumerate(group):
            # add points to the shape if the next shape has less points
            if len(shape["points"]) < len(frames[i + 1][j]["points"]):
                shape["points"] = add_substract_point(
                    [shape["points"], frames[i + 1][j]["points"]]
                )
            # get the difference between the points of the same shape in the next group
            difference = np.subtract(frames[i + 1][j]["points"], shape["points"])
            # get the step by which the points will be interpolated
            step.append(difference / frames_to_interpolate)
        # iterates over each shape in a group
        for k in range(1, frames_to_interpolate):
            for j, shape in enumerate(group):
                points = shape["points"].copy() + (step[j] * k)
                skelet["points"] = list(points)
                skelet["frame"] = shape["frame"] + k
                annots[0]["shapes"].insert(
                    indicies.get(shape["frame"]) + offset, copy.deepcopy(skelet)
                )
                offset += 1

In [46]:
interpolator(shapes_by_frame)


In [21]:
# with open('annotations.json') as json_file:
#     annots = json.load(json_file)


In [21]:
with open("test/annots/annotations_2pleura->fixed.json", "w") as aline_file:
    json.dump(annots, aline_file)
